In [1]:
!pip install tensorflow-addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 15.0 MB/s eta 0:00:0000:01
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.
ydata-profiling 4.12.2 requires typeguard<5,>=3, but you have typeguard 2.13.3 which is incompatible.


In [2]:
import glob
import os
import music21
import torch
import pickle 
import numpy as np
from keras.layers import LSTM, Dropout,Dense,Activation
from keras.layers import BatchNormalization 
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint

In [3]:
from music21 import  converter, instrument, note, chord
data_notes=[]

def load_data():
    for file in glob.glob("/kaggle/input/lakh-midi-clean/10cc/*.mid"):
        midi_file=converter.parse(file)
    print("parsing file%s" %file)
    notes_parsing= None
    try:
        s2=instrument.partitionByInstrument(midi_file)
        noets_parsing=s2.patrs[0].resurse()
    except:
        notes_parsing=midi_file.flat.notes
    for unit in notes_parsing:
        if isinstance(unit, note.Note):
            data_notes.append(str(unit.pitch))
        elif isinstance(unit, chord.Chord):
            data_notes.append('.'.join(str(n) for n in unit.normalOrder))

    with open('./data_notes', 'wb') as filepath:
        pickle.dump(data_notes, filepath)

    return data_notes
    
    

In [4]:
from tensorflow.keras.utils import to_categorical

def sequences(data_notes, n_vocab):
    sequence_length=100
    name_of_pitch = sorted(set(item for item in data_notes))
    note_to_int = dict((note, number) for number, note in enumerate(name_of_pitch))

    inputs_model = []
    output_model = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(data_notes) - sequence_length, 1):
        sequence_in = data_notes[i:i + sequence_length]
        sequence_out = data_notes[i + sequence_length]
        inputs_model.append([note_to_int[char] for char in sequence_in])
        output_model.append(note_to_int[sequence_out])

    n_patterns = len(inputs_model)

    # reshape the input into a format compatible with LSTM layers
    inputs_model = np.reshape(inputs_model, (n_patterns, sequence_length, 1))
    # normalize input
    inputs_model = inputs_model / float(n_vocab)

    output_model = to_categorical(output_model, num_classes=n_vocab)

    return (inputs_model, output_model)

In [5]:
notes = load_data()  
n_vocab = len(set(notes))
inputs_model, output_model = sequences(notes, n_vocab)

parsing file/kaggle/input/lakh-midi-clean/10cc/Dreadlock_Holiday.4.mid


/usr/local/lib/python3.10/dist-packages/music21/stream/base.py:3689: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


In [6]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [7]:
import tensorflow as tf

# Set up MirroredStrategy for multi-GPU training
strategy = tf.distribute.MirroredStrategy()

print(f"Using {strategy.num_replicas_in_sync} GPUs")

Using 2 GPUs


In [8]:
 from tensorflow.keras.optimizers import AdamW
with strategy.scope():
    def Neural_Network(inputs_model, n_vocab):
        model = Sequential()
        model.add(LSTM(
            512,
            input_shape=(inputs_model.shape[1], inputs_model.shape[2]),
            recurrent_dropout=0.3,
            return_sequences=True,
            dtype="float32"
        ))
        model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3, dtype="float32"))
        model.add(LSTM(512, dtype="float32"))
        model.add(BatchNormalization(dtype="float32"))
        model.add(Dropout(0.3))
        model.add(Dense(256, activation='relu', dtype="float32"))
        model.add(BatchNormalization(dtype="float32"))
        model.add(Dropout(0.3))
        model.add(Dense(n_vocab, activation='softmax', dtype="float32"))

        optimizer = AdamW(learning_rate=0.001, weight_decay=0.01)

        model.compile(loss='categorical_crossentropy', optimizer=optimizer)

        return model

    model = Neural_Network(inputs_model, n_vocab)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [9]:
def train_model(model, inputs_model, output_model):
    filepath = "/kaggle/working/my_model/weights-{epoch:02d}-{loss:.4f}.keras"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=1,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(inputs_model, output_model, epochs=200, batch_size=256, callbacks=callbacks_list)
    model.save(filepath)

In [10]:
def train_network():
    global model  # Use global model variable created inside strategy.scope()
    notes = load_data()
    n_vocab = len(set(notes))
    inputs_model, output_model = sequences(notes, n_vocab)
    
    with strategy.scope():  # Ensure model is created inside strategy
        model = Neural_Network(inputs_model, n_vocab)

    train_model(model, inputs_model, output_model)

In [11]:
train_network()

parsing file/kaggle/input/lakh-midi-clean/10cc/Dreadlock_Holiday.4.mid
Epoch 1/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - loss: 5.2091
Epoch 1: loss improved from inf to 4.98978, saving model to /kaggle/working/my_model/weights-01-4.9898.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 24s 336ms/step - loss: 5.2030
Epoch 2/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step - loss: 4.5174
Epoch 2: loss improved from 4.98978 to 4.40656, saving model to /kaggle/working/my_model/weights-02-4.4066.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 12s 336ms/step - loss: 4.5144
Epoch 3/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 335ms/step - loss: 4.1167
Epoch 3: loss improved from 4.40656 to 4.05366, saving model to /kaggle/working/my_model/weights-03-4.0537.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 12s 346ms/step - loss: 4.1150
Epoch 4/200
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - loss: 3.9042
Epoch 4: loss improved from 4.05366 to 3.85063, saving model to /kaggle/working/my_model/weights-04-3.8506.keras
35/35 ━━━━━━━━━━━━━━━━━━━━ 12s 335ms/s

In [59]:
import numpy as np
import pickle
import torch
from music21 import instrument, note, chord, stream

def generate_music(model, data_notes, n_vocab, output_length=500):
    # Load note mappings
    name_of_pitch = sorted(set(data_notes))
    note_to_int = {note: number for number, note in enumerate(name_of_pitch)}
    int_to_note = {number: note for note, number in note_to_int.items()}
    
    # Choose a random seed sequence
    sequence_length = 100
    start_index = np.random.randint(0, len(data_notes) - sequence_length - 1)
    seed_sequence = data_notes[start_index: start_index + sequence_length]
    
    # Convert seed sequence to integer format
    input_sequence = [note_to_int[note] for note in seed_sequence]

    # Generate music
    generated_notes = []
    for _ in range(output_length):
        # Ensure correct shape: (1, sequence_length, 1)
        input_data = np.reshape(input_sequence, (1, sequence_length, 1))
        input_data = input_data / float(n_vocab)  # Normalize

        #  FIX: Do NOT reshape beyond (1, 100, 1)
        predicted_probs = model.predict(input_data, verbose=0)
        predicted_index = np.argmax(predicted_probs)
        predicted_note = int_to_note[predicted_index]
        
        generated_notes.append(predicted_note)
        input_sequence.append(predicted_index)
        input_sequence = input_sequence[1:]  # Shift window

    # Convert to MIDI
    output_midi = convert_to_midi(generated_notes)
    output_midi.write('midi', fp='generated_music.mid')
    print("Music generated and saved as 'generated_music.mid'")


def convert_to_midi(notes):
    output_notes = []
    for pattern in notes:
        if '.' in pattern or pattern.isdigit():
            chord_notes = [note.Note(int(n)) for n in pattern.split('.')]
            new_chord = chord.Chord(chord_notes)
            new_chord.duration.quarterLength = 0.5
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.duration.quarterLength = 0.5
            output_notes.append(new_note)
    
    # Fix: Don't use output_notes[1], use full list
    midi_stream = stream.Stream(output_notes)
    return midi_stream



In [18]:
print(f"Total notes available: {len(data_notes)}")

Total notes available: 8870


In [49]:
print(model.input_shape)

(None, 100, 1)


In [61]:
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('/kaggle/working/my_model/weights-90-0.1243.keras')  # Ensure correct path

In [ ]:
model.summary()

In [63]:
import pickle

with open('/kaggle/working/data_notes', 'rb') as f:
    data_notes = pickle.load(f)

n_vocab = len(set(data_notes))


In [ ]:
generate_music(model, data_notes, n_vocab, output_length=500)

Music generated and saved as 'generated_music.mid'


In [ ]:
import pygame

pygame.mixer.init()

# Load and play the midi file
pygame.mixer.music.load("generated_music.mid")
pygame.mixer.music.play()

while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)  
